### Computing PCA using RDDs

###  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We want to perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### `nan`s in arithmetic operations
* We store all of the measurements as single `bytearray` in a single column. Instead of using 365 columns.
* Why?
  1. Because serializing and desirializing is faster that way.
  1. Because numpy treats `nan` entries correctly:
      * In `numpy.nansum` `5+nan=5` while in dataframes `5+nan=nan`

In [1]:
import numpy as np
X=np.array([1,1,1,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)
X=np.array([1,1,np.NaN,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)

mean of [1 1 1 2] = 1.25
nanmean of [1 1 1 2] = 1.25
mean of [  1.   1.  nan   2.] = nan
nanmean of [  1.   1.  nan   2.] = 1.33333333333


#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

In [2]:
x1=np.array([1,np.NaN,3,4,5])
x2=np.array([2,3,4,np.NaN,6])
stacked=np.array([np.outer(x1,x1),np.outer(x2,x2)])
stacked

array([[[  1.,  nan,   3.,   4.,   5.],
        [ nan,  nan,  nan,  nan,  nan],
        [  3.,  nan,   9.,  12.,  15.],
        [  4.,  nan,  12.,  16.,  20.],
        [  5.,  nan,  15.,  20.,  25.]],

       [[  4.,   6.,   8.,  nan,  12.],
        [  6.,   9.,  12.,  nan,  18.],
        [  8.,  12.,  16.,  nan,  24.],
        [ nan,  nan,  nan,  nan,  nan],
        [ 12.,  18.,  24.,  nan,  36.]]])

In [3]:
np.nanmean(stacked,axis=0)

/Users/egouda/anaconda2/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


array([[  2.5,   6. ,   5.5,   4. ,   8.5],
       [  6. ,   9. ,  12. ,   nan,  18. ],
       [  5.5,  12. ,  12.5,  12. ,  19.5],
       [  4. ,   nan,  12. ,  16. ,  20. ],
       [  8.5,  18. ,  19.5,  20. ,  30.5]])

### Loading Data into Dataframe

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py']) #,'lib/spark_PCA.py'

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [2]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
# add after creating spark_PCA.py
# from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into 256 geographical rectangles, indexed from BBBBBBBB to SSSSSSSS. And each containing about 12,000 station,year pairs.

In [3]:
file_index='SBSSBSBB'
filename='US_Weather_%s.csv.gz'%file_index
data_dir='../../Data/Weather'
del_files='%s/%s*'%(data_dir,filename[:-3])
print del_files
!rm $del_files
command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(filename,data_dir,filename)
print command
!$command
!ls -lh $data_dir/$filename

../../Data/Weather/US_Weather_SBSSBSBB.csv*
curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_SBSSBSBB.csv.gz > ../../Data/Weather/US_Weather_SBSSBSBB.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3726k  100 3726k    0     0  2892k      0  0:00:01  0:00:01 --:--:-- 3017k
-rw-r--r--  1 egouda  staff   3.6M May  8 23:32 ../../Data/Weather/US_Weather_SBSSBSBB.csv.gz


In [4]:
!gunzip $data_dir/$filename
filename=data_dir+'/US_Weather_SBSSBSBB.csv'
!ls -lh $filename

-rw-r--r--  1 egouda  staff    13M May  8 23:32 ../../Data/Weather/US_Weather_SBSSBSBB.csv


In [5]:
import pickle
List=pickle.load(open(filename,'rb'))
len(List)

12619

In [10]:
df=sqlContext.createDataFrame(List)
print df.count()
df.show(5)
outfilename="US_Weather_SBSSBSBB.parquet"
#!rm -rf $dir/$outfilename
df.write.save(outfilename)
#!ls -ld $dir/$outfilename

12619
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|   1451.5| 42.0358|-111.9617|       TMAX|USC00109698|    42|[40 4C 80 4D 80 5...|1899.0|SBSSBSBB|
|   1451.5| 42.0358|-111.9617|       TMAX|USC00109698|    17|[80 D1 80 C9 E0 5...|1900.0|SBSSBSBB|
|   1451.5| 42.0358|-111.9617|       TMAX|USC00109698|    16|[A0 D3 00 00 00 5...|1901.0|SBSSBSBB|
|   1451.5| 42.0358|-111.9617|       TMAX|USC00109698|    37|[80 54 E0 50 00 4...|1902.0|SBSSBSBB|
|   1451.5| 42.0358|-111.9617|       TMAX|USC00109698|    21|[00 C6 00 46 00 4...|1903.0|SBSSBSBB|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



In [11]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [12]:
from time import time
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
print 'took',time()-t,'seconds'

Number of executors= 3
took 0.00147414207458 seconds


### Computing PCA for each measurement

In [13]:
measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']

#### Homework
The code below computes the covariance matrix using RDDs.
The code allows undefined entries and calculates the covariance without bias.

Your homework is to complete the missing parts in the code (Marked with `...`) so that it would calculate the covariance correctly.

In [14]:
#%%writefile lib/spark_PCA.py #once this works correctly, you should add it to the `lib` directory
# and use import

import numpy as np
from numpy import linalg as LA

def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O=np.outer(X,X)
    N=1-np.isnan(O)
    return (O,N)

def sumWithNan(M1,M2):
    """Add two pairs of (matrix,count)"""
    (X1,N1)=M1
    (X2,N2)=M2
    N=N1+N2
    X=np.nansum(np.dstack((X1,X2)),axis=2)
    return (X,N)

def computeCov(RDDin):
    """computeCov recieves as input an RDD of np arrays, all of the same length, 
    and computes the covariance matrix for that set of vectors"""
    RDD=RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) 
                     # insert a 1 at the beginning of each vector so that the same 
                     #calculation also yields the mean vector
     # separating the map and the reduce does not matter because of Spark uses lazy execution.
    OuterRDD=RDD.map(outerProduct)   
    (S,N)=OuterRDD.reduce(sumWithNan)
    E=S[0,1:]
    NE=np.float64(N[0,1:])
    print 'shape of E=',E.shape,'shape of NE=',NE.shape
    Mean=E/NE
    O=S[1:,1:]
    NO=np.float64(N[1:,1:])
    Cov=O/NO - np.outer(Mean,Mean)
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}

In [15]:
from numpy import linalg as LA
STAT={}  # dictionary storing the statistics for each measurement
Clean_Tables={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM weather\n\tWHERE measurement='%s'"%(meas)
    #Query="SELECT * FROM weather WHERE measurement='"+meas+"'"
    print Query
    df1 = sqlContext.sql(Query)
    data=df1.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
    #get basic statistics
    STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

    # compute covariance matrix
    OUT=computeCov(data)

    #find PCA decomposition
    eigval,eigvec=LA.eig(OUT['Cov'])

    # collect all of the statistics in STAT[meas]
    STAT[meas]['eigval']=eigval
    STAT[meas]['eigvec']=eigvec
    STAT[meas].update(OUT)

    print 'time for',meas,'is',time()-t

SELECT * FROM weather
	WHERE measurement='TMAX'
shape of E= (365,) shape of NE= (365,)
time for TMAX is 16.0453760624
SELECT * FROM weather
	WHERE measurement='SNOW'
shape of E= (365,) shape of NE= (365,)
time for SNOW is 12.5414819717
SELECT * FROM weather
	WHERE measurement='SNWD'
shape of E= (365,) shape of NE= (365,)
time for SNWD is 10.1639611721
SELECT * FROM weather
	WHERE measurement='TMIN'
shape of E= (365,) shape of NE= (365,)
time for TMIN is 14.5669820309
SELECT * FROM weather
	WHERE measurement='PRCP'
shape of E= (365,) shape of NE= (365,)
time for PRCP is 15.1397848129
SELECT * FROM weather
	WHERE measurement='TOBS'
shape of E= (365,) shape of NE= (365,)
time for TOBS is 8.66682910919


In [20]:
from pickle import dump
filename=data_dir+'/STAT_%s.pickle'%file_index
dump((STAT,STAT_Descriptions),open(filename,'wb'))
!ls -lrth $data_dir

total 441120
-rw-r--r--    1 egouda  staff   2.1M Apr 29 14:50 stations.pkl.gz
-rw-r--r--    1 egouda  staff   270B Apr 29 14:50 ghcnd-version.txt
-rw-r--r--    1 egouda  staff   7.4M Apr 29 14:50 ghcnd-stations_buffered.txt
-rw-r--r--    1 egouda  staff   8.0M Apr 29 14:50 ghcnd-stations.txt
-rw-r--r--    1 egouda  staff    22K Apr 29 14:50 ghcnd-readme.txt
drwxr-xr-x  404 egouda  staff    13K Apr 29 14:50 decon_BBBSBBBB_TOBS.parquet
drwxr-xr-x  404 egouda  staff    13K Apr 29 14:50 decon_BBBSBBBB_SNWD.parquet
drwxr-xr-x  404 egouda  staff    13K Apr 29 14:50 decon_BBBSBBBB.parquet
-rw-r--r--    1 egouda  staff   218B Apr 29 14:50 data-source.txt
drwxr-xr-x   10 egouda  staff   340B Apr 29 14:50 US_Weather_BBBSBBBB.parquet
-rw-r--r--    1 egouda  staff    16M Apr 29 14:50 STAT_BBBSBBBB.pickle.gz
-rw-r--r--    1 egouda  staff    78M Apr 29 15:05 STAT_BBBSBBBB.pickle
-rw-r--r--    1 egouda  staff    13M May  8 22:02 US_Weather_BBBSBBBB.csv
-rw-r--r--    1 egouda  staff    13M May  8 23:

In [17]:
import pandas as pd

In [18]:
data_df_1=pd.DataFrame(STAT)

In [19]:
data_df_1

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS
Cov,"[[1255.18704999, 180.216970881, 292.120831212,...","[[717.988099289, 37.8653805396, 12.3798448324,...","[[20284.5000357, 19630.4494596, 19463.3041104,...","[[2168.36994376, 1593.61014039, 1067.78015168,...","[[3815.00448132, 2186.87904021, 1365.37318824,...","[[3188.33736181, 1774.86109913, 1228.9956602, ..."
E,"[30271.0, 38085.0, 29351.0, 30899.0, 47258.0, ...","[14901.0, 21240.0, 12171.0, 13261.0, 20343.0, ...","[140689.0, 152046.0, 149857.0, 151133.0, 15756...","[5108.0, 5426.0, -1076.0, 3632.0, 7188.0, 7260...","[-256604.0, -252349.0, -262361.0, -253544.0, -...","[-85153.0, -89408.0, -91819.0, -85300.0, -8268..."
Mean,"[12.3858428805, 15.4754164974, 11.9264526615, ...","[9.39533417402, 13.4006309148, 7.69342604298, ...","[122.019947962, 130.736027515, 129.075796727, ...","[2.09515996719, 2.2237704918, -0.440802949611,...","[-106.784852268, -106.028991597, -108.77321724...","[-44.3967674661, -46.4940197608, -48.427742616..."
NE,"[2444.0, 2461.0, 2461.0, 2460.0, 2463.0, 2470....","[1586.0, 1585.0, 1582.0, 1580.0, 1585.0, 1587....","[1153.0, 1163.0, 1161.0, 1164.0, 1161.0, 1167....","[2438.0, 2440.0, 2441.0, 2417.0, 2439.0, 2447....","[2403.0, 2380.0, 2412.0, 2366.0, 2401.0, 2391....","[1918.0, 1923.0, 1896.0, 1906.0, 1922.0, 1922...."
NO,"[[2444.0, 2431.0, 2425.0, 2423.0, 2423.0, 2430...","[[1586.0, 1557.0, 1551.0, 1548.0, 1548.0, 1550...","[[1153.0, 1134.0, 1126.0, 1122.0, 1117.0, 1124...","[[2438.0, 2384.0, 2364.0, 2328.0, 2342.0, 2354...","[[2403.0, 2328.0, 2323.0, 2274.0, 2294.0, 2295...","[[1918.0, 1855.0, 1813.0, 1809.0, 1815.0, 1822..."
O,"[[3442609.0, 904072.0, 1066612.0, 1513910.0, 1...","[[1278729.0, 254988.0, 131311.0, 149820.0, 223...","[[40554893.0, 40350955.0, 39649980.0, 38896730...","[[5297188.0, 3810274.0, 2522049.0, 1779656.0, ...","[[36568876.0, 31449346.0, 30154178.0, 28513076...","[[9895749.0, 7121429.0, 6126183.0, 5160870.0, ..."
SortedVals,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-106.0, -72.0, -67.0, -56.0, -44.0, -39.0, -1...","[-239.0, -211.0, -200.0, -183.0, -178.0, -172....","[-139.0, -122.0, -100.0, -89.0, -67.0, -56.0, ..."
UnDef,"[1, 0, 0, 0, 0, 0, 0, 0, 39, 1, 1, 1, 1, 0, 2,...","[1, 9, 0, 30, 31, 1, 30, 1, 0, 31, 1, 1, 2, 7]","[3, 25, 3, 29, 18, 0, 6, 1, 0, 4, 44, 0, 0, 26...","[4, 39, 4, 6, 0, 1, 4, 41, 2, 16, 3, 0, 0, 4, ...","[2, 3, 0, 12, 0, 4, 5, 8, 35, 0, 0, 5, 7, 3, 5...","[5, 31, 1, 1, 5, 32, 5, 4, 34, 39, 0, 38, 0, 4..."
Var,"[1255.18704999, 1597.70165083, 1211.65367246, ...","[717.988099289, 1146.60100907, 493.00778232, 6...","[20284.5000357, 21606.4006539, 20285.1294831, ...","[2168.36994376, 2453.46550054, 2571.80978944, ...","[3815.00448132, 4101.51638638, 4275.94649661, ...","[3188.33736181, 3084.9915139, 3178.28697296, 3..."
eigval,"[26620.487435, 14924.0987613, 12967.360107, 11...","[3258.50039568, 2332.71326161, 2085.05861248, ...","[2049400.84042, 257658.279576, 219215.327871, ...","[249783.500923, 40763.3386597, 32250.0127624, ...","[270547.314847, 33050.5969039, 24441.3598997, ...","[735681.763614, 44518.9530609, 29109.3100455, ..."
